In [99]:
#import dependencies 
import pandas as pd 
import requests 
from sqlalchemy import create_engine
import psycopg2
import config 
import queries
from datetime import datetime 


In [106]:
200000 * 31

6200000

In [ ]:
# get month and loop through each day ( will need current day and day after as between range)
datetime.month.date

In [96]:
#ampersand to chain filters
#must set limit above expected data or default limt set to 1000
api_url2 = "https://data.sfgov.org/resource/imvp-dq3v.json?$limit=3000&$where=session_start_dt between '2022-08-01' and '2022-09-01'"


In [118]:
api_url = "https://data.sfgov.org/resource/imvp-dq3v.json?$limit=100000&$where=session_start_dt between '2022-08-01' and '2022-08-02'"


In [29]:
class GetData():
    #api request
     def __init__(self, endpoint_url):
        print("requesting data")
        self.endpoint = endpoint_url
        self.response = requests.get(url=self.endpoint)
        self.data = self.response.json() 
        print("data request complete")
    

In [93]:
test1 = GetData(api_url2)

requesting data
data request complete


In [124]:
test2 = GetData(api_url)

requesting data
data request complete


In [125]:
#test daily transactions size
(len(test2.data) + 20000) * 31

2646842